In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss

import acr.subjects as subs
import acr
import plotly.express as px

In [ ]:
a12 = {}
a12["subject"] = "ACR_12"
a12["complete_key_list"] = ["control1-bl"]
a12["paths"] = acr.io.get_acr_paths(a12["subject"], a12["complete_key_list"])
a12["load_times"] = {}
a12["load_times"]["control1-bl"] = (0, 1)
a12["load_times"]["laser1-bl"] = (0, 0)
a12["load_times"]["control1"] = (0, 0)
a12["load_times"]["laser1"] = (0, 0)
a12["channels"] = {}
a12["channels"]["EEGr"] = [1, 2]
a12["channels"]["NNXr"] = [3, 14]
a12["channels"]["LFP_"] = [3, 14]


In [ ]:
data, spg, bp = acr.io.acr_load_master(a12, type='pandas', stores=['NNXr', 'LFP_'], hyp=False)

In [ ]:
datax, spgx, bpx = acr.io.acr_load_master(a12, type='xarray', stores=['NNXr', 'LFP_'], hyp=False)

In [ ]:
type(bp['control1-bl-LFP_'])

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(20, 10))
ax[0].plot(bp['control1-bl-LFP_'].xc(3).dt(), bp['control1-bl-LFP_'].xc(3).sigma)
ax[0].plot(bp['control1-bl-NNXr'].xc(3).dt(), bp['control1-bl-NNXr'].xc(3).sigma)
ax[1].plot(bp['control1-bl-LFP_'].xc(14).dt(), bp['control1-bl-LFP_'].xc(14).sigma)
ax[1].plot(bp['control1-bl-NNXr'].xc(14).dt(), bp['control1-bl-NNXr'].xc(14).sigma)
ax[0].set_ylim(0, 100000)

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(20, 10))
ax[0].plot(bp['control1-bl-LFP_'].xc(3).dt(), bp['control1-bl-LFP_'].xc(3).delta)
ax[0].plot(bp['control1-bl-NNXr'].xc(3).dt(), bp['control1-bl-NNXr'].xc(3).delta)
ax[1].plot(bp['control1-bl-LFP_'].xc(14).dt(), bp['control1-bl-LFP_'].xc(14).delta)
ax[1].plot(bp['control1-bl-NNXr'].xc(14).dt(), bp['control1-bl-NNXr'].xc(14).delta)
ax[0].set_ylim(0, 100000)

In [ ]:
import scipy.signal as signal


In [ ]:
def decimate(sig, q=5):
    dat = signal.decimate(sig.values, q=q, ftype="fir", axis=0)
    rs = xr.DataArray(
        dat,
        dims=sig.dims,
        coords={
            **sig["datetime"][::q].coords,
            **sig[sig.dims[-1]].coords,
        },
        attrs=sig.attrs,
    )
    rs.attrs["fs"] = sig.fs / q
    return rs

In [ ]:
ds = datax['control1-bl-NNXr']

In [ ]:
rs = decimate(ds, q=122)

In [ ]:
rs_dict = {}
rs_dict['c14'] = rs

In [ ]:
srs, brs = acr.io.get_spectral(rs_dict)

In [ ]:
brs['c14']

In [ ]:
d = data['control1-bl-NNXr']

In [ ]:
@xr.register_dataset_accessor("ex")
@xr.register_dataarray_accessor("ex")
class xraccessor:
    def __init__(self, xarray_obj):
        self._obj = xarray_obj

    def is_coordinate_on_dimension(self, coord, dim):
        return coord in self._obj[dim].coords.keys()

    def get_dimension(self, coord):
        matching_dims = [
            dim for dim in self._obj.dims if self.is_coordinate_on_dimension(coord, dim)
        ]
        assert (
            len(matching_dims) == 1
        ), "Selection coordinate must be present on exatly one dimension."
        return matching_dims[0]
    def xc(self, chan):
        return self._obj.sel(channel=chan)

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(20, 10))
ax[0].plot(brs['c14'].datetime, brs['c14'].sel(channel=3).delta)
ax[0].plot(bp['control1-bl-NNXr'].xc(3).dt(), bp['control1-bl-NNXr'].xc(3).delta)
ax[0].set_ylim(0, 100000)

In [ ]:
t_end = [40, 60, 80]
paths = ['p1', 'p2', 'p3']

def create_string(st, intg):
    return st + str(intg)

recs = [create_string(p, end) for p, end in zip(paths, t_end)]
recs

In [ ]:
ferrule_data = pd.read_csv(p, header=18, sep=';')

In [ ]:
ferrule_data

In [ ]:
# convert all commas to dots
ferrule_data = ferrule_data.replace(',', '.', regex=True)
ferrule_data['Power (W)'] = ferrule_data['Power (W)'].astype(float)
ferrule_data

In [ ]:
def read_ferrule_data(path):
        """
        Reads the ferrule data from the specified path (single .csv file).
        """

        path = Path(path)
        
        # Read the ferrule data
        ferrule_data = pd.read_csv(path, header=18, sep=';')
        ferrule_data = ferrule_data.replace(',', '.', regex=True).astype(float)

        # Convert to seconds and microWatts
        ferrule_data['Power (W)'] = ferrule_data['Power (W)'] * 1e6
        ferrule_data['Time (ms)'] = ferrule_data['Time (ms)']/1e3

        # rename the columns with the correct units
        ferrule_data.rename(columns={'Time (ms)': 'Time (s)', 'Power (W)': 'Power (uW)'}, inplace=True)

        #add a column for the ferrule ID
        ferrule_data['Ferrule'] = path.stem.split('-')[0]

        #add a column for the patch-cord identifier (or fiber split)
        ferrule_data['Fiber'] = path.stem.split('-')[1]

        # add a column for the knob value
        ferrule_data['Knob'] = path.stem.split('-')[-1]

        return ferrule_data

In [ ]:
p = '/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/ferrules/f1-pc1-9.0-2.41-ss-0.5hz(warm1).csv'
data = read_ferrule_data(p)

In [146]:
os.path.split(p)

('/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/ferrules',
 'f1-pc1-9.0-2.41-ss-0.5hz(warm1).csv')

In [149]:
def get_ferrule_names(root):
    root = Path(root)
    files = [str(p) for p in root.glob("*.csv")]
    ferrule_names = set()
    for f in files:
        f = Path(f)
        ferrule_names.add(os.path.split(f)[1])
    return ferrule_names

get_ferrule_names('/Volumes/opto_loc/Data/ACR_PROJECT_MATERIALS/ferrules/')

{'f1-pc1-9.0-2.41-bttn-na(post_warmup).csv',
 'f1-pc1-9.0-2.41-ss-0.5hz(short_post_warmup)-1.0amp-2.2offset.csv',
 'f1-pc1-9.0-2.41-ss-0.5hz(short_post_warmup).csv',
 'f1-pc1-9.0-2.41-ss-0.5hz(short_pre_warmup).csv',
 'f1-pc1-9.0-2.41-ss-0.5hz(warm1).csv',
 'f1-pc1-9.0-2.41-ss-0.5hz(warm2).csv',
 'f1-pc1-9.0-2.41-ss-2hz(post_warmup).csv',
 'f1-pc1-9.0-2.41-ttl-2hz(post_warmup)-0.92amp-2.2offset.csv',
 'f1-pc1-9.0-2.41-ttl-2hz(post_warmup)-part2.csv',
 'f1-pc1-9.0-2.41-ttl-2hz(post_warmup).csv',
 'f1-pc1-9.0-2.41-ttl-2hz(pre_warmup).csv'}

In [ ]:
px.line(data, x='Time (s)', y='Power (uW)')